# Starter notebook - Reading files

In [0]:
from pyspark.sql.types import *

from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("my project 1").getOrCreate()
sc = spark.sparkContext

# Read a CSV into a dataframe
# There is a smarter version, that will first check if there is a Parquet file and use it
def load_PD_file(filename_or_dir, schema) :
    dataPath = "/mnt/ddscoursedatastorage/fwm-stb-data/" + filename_or_dir
    df = spark.read.format("csv")\
      .option("header","false")\
      .option("delimiter", "|")\
      .schema(schema)\
      .load(dataPath)
    return df

In [0]:
df1 = spark.read.csv("/mnt/ddscoursedatastorage/dds-students/test.csv")

# Reading the data files

## Reading Refrence Data

In [0]:
# Reading the Reference Parquet files

ref_data = spark.read.parquet('/ref_data_raw').withColumnRenamed("_device-id","device_id")\
                                                .withColumnRenamed("_dma","dma")\
                                                .withColumnRenamed("_dma-code","dma_code")\
                                                .withColumnRenamed("_household-id","household_id")\
                                                .withColumnRenamed("_household-type","household_type")\
                                                .withColumnRenamed("_system-type","system_type")\
                                                .withColumnRenamed("_zipcode","zipcode")
# ref_data_count = ref_data.count()
# print(ref_data_count)
# ref_data.limit(5).show()

## Reading Daily Program Data

In [0]:

# Reading the Daily Programs CSV file

daily_prog_schema =  StructType([StructField('prog_code',StringType()),
                     StructField('title',StringType()),
                     StructField('genre',StringType()),
                     StructField('air_date',StringType()),
                     StructField('air_time',StringType()),
                     StructField('Duration',FloatType())
                                       ])
daily_prog_data = load_PD_file("Daily program data/" , daily_prog_schema  )

# daily_prog_data.limit(3).show()

## Reading Program Viewing Data

In [0]:
#Reading the 2.5% sample of the viewing data from a Parquet file
 
viewing_data = spark.read.parquet('/sample_viewing_2_5percent')
 
# print(f'There are {viewing_data.count():,} entries in viewing_data dataframe!')
# viewing_data.show(5)

## Reading Demographic Data

In [0]:

# Reading the Demographic CSV file

demographic_schema =  StructType([StructField('household_id',StringType()),
                      StructField('household_size',IntegerType()),
                      StructField('num_adults',IntegerType()),
                      StructField('num_generations',IntegerType()),
                      StructField('adult_range',StringType()),
                      StructField('marital_status',StringType()),
                      StructField('race_code',StringType()),
                      StructField('presence_children',StringType()),
                      StructField('num_children',IntegerType()),
                      StructField('age_children',StringType()), #format like range - 'bitwise'
                      StructField('age_range_children',StringType()),
                      StructField('dwelling_type',StringType()),
                      StructField('home_owner_status',StringType()),
                      StructField('length_residence',IntegerType()),
                      StructField('home_market_value',StringType()),
                      StructField('num_vehicles',IntegerType()),
                      StructField('vehicle_make',StringType()),
                      StructField('vehicle_model',StringType()),
                      StructField('vehicle_year',IntegerType()),
                      StructField('net_worth',IntegerType()),
                      StructField('income',StringType()),
                      StructField('gender_individual',StringType()),
                      StructField('age_individual',IntegerType()),
                      StructField('education_highest',StringType()),
                      StructField('occupation_highest',StringType()),
                      StructField('education_1',StringType()),
                      StructField('occupation_1',StringType()),
                      StructField('age_2',IntegerType()),
                      StructField('education_2',StringType()),
                      StructField('occupation_2',StringType()),
                      StructField('age_3',IntegerType()),
                      StructField('education_3',StringType()),
                      StructField('occupation_3',StringType()),
                      StructField('age_4',IntegerType()),
                      StructField('education_4',StringType()),
                      StructField('occupation_4',StringType()),
                      StructField('age_5',IntegerType()),
                      StructField('education_5',StringType()),
                      StructField('occupation_5',StringType()),
                      StructField('polit_party_regist',StringType()),
                      StructField('polit_party_input',StringType()),
                      StructField('household_clusters',StringType()),
                      StructField('insurance_groups',StringType()),
                      StructField('financial_groups',StringType()),
                      StructField('green_living',StringType())
                                       ])

demographic_data = load_PD_file("demographic/" , demographic_schema  )  

# display(demographic_data.limit(1))

# 1. Spam Detection

## 1.1 Extract and Transform

### Assumptions
Due to the fact that the primary key in the Refrence Data relation is (household_id, device_id), and in the viewing relation there is no household_id attribute, joining both relations in order to get information on views per device will result in erroneuos data. to overcome this issue, we filtered the refrence data by taking the first household thats ascribed to each device_id.

### Pre-Proccessing the data

In [0]:
import pyspark.sql.functions as f

In [0]:
# filter unwanted colmuns from all the data sets.
#GPT
from pyspark.sql.functions import window, column, desc, col, when, to_timestamp, to_date

ref_filtered= ref_data.select(['device_id','dma','household_id'])\
                                .dropDuplicates()\
                                    .groupBy(["device_id",'dma'])\
                                        .agg(f.first('household_id').alias("household_id"))
daily_prog_filterd=daily_prog_data.select("prog_code","genre","Duration","air_time","air_date").dropDuplicates()
viewing_filterd=viewing_data.select("prog_code","device_id","event_time", "event_date").dropDuplicates()
demographic_filterd=demographic_data.select("household_id","net_worth","num_adults","household_size","income").dropDuplicates()

## joinning all DataFrames

In [0]:
# can't actually perform this join as its too heavy computatuionally
full_join = ref_filtered.join(viewing_filterd, on='device_id')\
                            .join(daily_prof_filterd, on='prog_code')\
                                .join(demographic_filterd, 'household_id')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1544834338005280> in <cell line: 2>()
      1 # can't actually perform this join as its too heavy computatuionally
----> 2 full_join = ref_filtered.join(viewing_filterd, on='device_id')\
      3                             .join(daily_prof_filterd, on='prog_code')\
      4                                 .join(demographic_filterd, 'household_id')

NameError: name 'ref_filtered' is not defined

## 1.2  Computation and Detection

### Query1
 The prog code was viewed by a device with less than 5 average daily events.

In [0]:

viewing_filterd1 = viewing_filterd\
        .na.drop(how='any', subset = ['device_id', 'event_date','event_time','prog_code'])
from pyspark.sql.functions import window, column, desc, col
viewing_filterd1.createOrReplaceTempView("viewing_filter")
query1 = spark.sql("""         SELECT distinct device_id 
                               FROM viewing_filter
                               GROUP BY device_id 
                               having count(*)/ COUNT(DISTINCT event_date) <5
                               """)
query1_prog_code=viewing_filterd1.join(query1,on="device_id").select("prog_code").distinct()
query1_prog_code = query1_prog_code.withColumnRenamed("prog_code", "query1")
query1_prog_code.write.mode("overwrite").parquet("part1_q1_progs_full") ### saved ##
num=query1_prog_code.count()
print(f"there are {num} programs that match the criteria ")


there are 9138 programs that match the criteria 


### Query2
The prog code was watched by a device from a DMA name that contains the letter [‘z’] (case
insensitive).

In [0]:
ref_filtered2 = ref_filtered.dropna(how = 'any', subset =['device_id', 'dma','household_id'])
ref_filtered2.createOrReplaceTempView("ref_filtered2")
query2=spark.sql("""SELECT distinct r.device_id 
                               FROM ref_filtered2 r 
                               WHERE  r.DMA LIKE '%z%' or r.DMA LIKE '%Z%'"""
                               ) 
query2_prog_code= viewing_filterd.join(query2, on="device_id").select("prog_code").distinct()                          
query2_prog_code = query2_prog_code.withColumnRenamed("prog_code", "query2")
query2_prog_code.write.mode("overwrite").parquet("part1_q2_progs_full") ### saved ##
num=query2_prog_code.count()
print(f"there are {num} programs that match the criteria ")


there are 235428 programs that match the criteria 


### Query 3
The program was watched by a family with less than 3 adults and their net worth is higher than 8
(both exclusive).

In [0]:
demographic_filterd3 = demographic_filterd.dropna(how = 'any', subset =['net_worth', 'num_adults'])
demographic_filterd3.createOrReplaceTempView("demographic_filterd3")
query3=spark.sql(""" SELECT  distinct household_id 
                     FROM demographic_filterd3
                     WHERE num_adults < 3 AND net_worth > 8;
                            """   ) 
                               
query3_prog_code= ref_filtered.join(query3, on="household_id").select("device_id").join(viewing_filterd, on="device_id").select("prog_code").distinct()
query3_prog_code = query3_prog_code.withColumnRenamed("prog_code", "query3")
query3_prog_code.write.mode("overwrite").parquet("part1_q3_progs_full") ### saved ##
num=query3_prog_code.count()
print(f"there are {num} programs that match the criteria ")



there are 179867 programs that match the criteria 


### Query 4
The program code was aired (at least once) between Friday at 6PM and Saturday at 7PM (both
inclusive) and there was at-least one household who watched the program with size higher than or
equal to 8 (inclusive).

In [0]:
from pyspark.sql.functions import col, hour, dayofweek, to_timestamp, to_timestamp, to_date
#clean the DFs # GPT
daily_prog_filtered_4= daily_prog_filterd.dropna(how = 'any', subset =['air_time', 'air_date'])\
                                .withColumn("eventTime",to_timestamp(col("air_time"),"HHmmss"))\
                                .drop(col("event_time"))\
                                .withColumn("eventDate", to_date(col("air_date"), "yyyyMMdd"))\
                                    .drop(col("event_date"))
                                            
demographic_filtered_4= demographic_filterd.dropna(how = 'any', subset =['household_size'])

progs_between_4_6 = daily_prog_filtered_4.filter(\
    (dayofweek(col("eventDate")) == f.lit(6)) & (hour(col("eventTime")) >= f.lit(180000))\
    | ((dayofweek(col("eventDate")) == f.lit(7)) & (hour(col("eventTime")) <= f.lit(190000))))\
         .select(col("prog_code")).distinct()
more_than_8_house = demographic_filtered_4.filter(demographic_filtered_4["household_size"]>=8)\
                                 .select(col("household_id"))
more_than_8_progs = more_than_8_house.join(ref_filtered,"household_id")\
                                        .join(viewing_filterd,"device_id")\
                                            .select("prog_code").distinct()
query_4_progs = more_than_8_progs.intersect(progs_between_4_6).distinct()
query_4_progs = query_4_progs.withColumnRenamed("prog_code", "query4")
query_4_progs.write.mode("overwrite").parquet("part1_q4_progs_full") ### saved ##
num=query_4_progs.count()
print(f"there are {num} programs that match the criteria ")




there are 66047 programs that match the criteria 


### Query 5
The prog code was watched (at least once) by a device from an household with more than 3 devices
(exclusive) and the income of that household is lower than the average household income in the
data.

In [0]:
from pyspark.sql.functions import avg,countDistinct
demographic_filtered_5 = demographic_filterd.dropna(subset=['income'])
dem_data_clean = demographic_filtered_5.select("household_id","income").dropDuplicates()

need_change=["A","B","C","D"]
for i in range(len(need_change)):
    dem_data_clean = dem_data_clean.withColumn("income", when(col("income")==need_change[i], int(10+i)).otherwise(col("income")))

dem_data_clean = dem_data_clean.withColumn("income", col('income').cast('integer'))


average_income = dem_data_clean.select(avg(col("income"))).first()[0]
print(average_income)

device_count_df = ref_filtered.groupBy("household_id")\
    .agg(countDistinct("device_id").alias("device_count"))\
                    .filter(col("device_count")>f.lit(3))\
                        .join(dem_data_clean,"household_id")\
                            .filter(col("income")<average_income)\
                                .select("household_id")\
                                    .join(ref_filtered,"household_id")\
                                        .join(viewing_filterd,"device_id")\
                                            .select(col("prog_code")).distinct()

query5_prog_code = device_count_df.withColumnRenamed("prog_code", "query5")
query5_prog_code.write.mode("overwrite").parquet("part1_q5_progs_full") ### saved ##
num=query5_prog_code.count()
print(f"there are {num} programs that match the criteria ")


6.715162771873656
there are 253554 programs that match the criteria 


### Query 6
The program contains at least one of the genres [‘Talk’, ‘Politics’, ‘News’, ‘Community’, ‘Crime’]
and has a duration of more than 35 minutes (exclusive).

In [0]:

from pyspark.sql.functions import col, when

daily_prog_filterd6=daily_prog_data.select("prog_code","genre","Duration")
daily_prog_filterd6 = daily_prog_filterd6.na.drop(how='any',subset=['genre','Duration']).withColumn('genres', f.explode(f.split(f.col('genre'), ','))).drop('genre')
daily_prog_filterd6.createOrReplaceTempView("daily_prog_filterd6")
query6=spark.sql(""" SELECT DISTINCT prog_code
                     FROM daily_prog_filterd6
                     WHERE (genres IN ('Talk', 'Politics', 'News', 'Community', 'Crime')) AND (Duration > 35);
                            """   )

query6_prog_code = query6.withColumnRenamed("prog_code", "query6")
query6_prog_code.write.mode("overwrite").parquet("part1_q6_progs_full") ### saved ##
num=query6_prog_code.count()
print(f"there are {num} programs that match the criteria ")


there are 33648 programs that match the criteria 


### Final Output:

In [0]:
# retreiving all the malicious programs
from pyspark.sql.functions import col, when
all_prog_codes = viewing_filterd.select("prog_code").distinct()

final_table = all_prog_codes
queries = [spark.read.parquet(f'/part1_q{i+1}_progs_full').alias(f"query{i+1}_prog_code")\
             for i in range(6)]
query_columns = ["query1", "query2", "query3", "query4", "query5", "query6"]

for i, query_prog_code in enumerate(queries):
    join_condition = final_table["prog_code"] == query_prog_code[query_columns[i]]
    final_table = final_table.join(query_prog_code, join_condition, how="left")
    final_table = final_table.withColumn(query_columns[i], when(col(query_columns[i]).isNull(), 0).otherwise(1))


malicious_records = final_table.withColumn("should_be_returned?", col("query1") + col("query2") + col("query3") + col("query4") + col("query5") + col("query6")).filter(col("should_be_returned?")>3).select("prog_code")
malicious_records.write.mode('overwrite').parquet("malicious_records_5_6_full.parquet") ### saved ##




In [0]:
# adding their title and genres
from pyspark.sql.functions import countDistinct
final_df = spark.read.parquet("/malicious_records_5_6_full.parquet")
filtered_for_result=daily_prog_data.select("prog_code", "title", "genre")
final_df=final_df.join(filtered_for_result, on="prog_code",how="left")
distinct_df = final_df.dropDuplicates(['prog_code'])
distinct_df.write.mode("overwrite").csv("final_answer_part1")


display(distinct_df)

prog_code,title,genre
EP000000211659,20/20,Newsmagazine
EP000000211661,20/20,Newsmagazine
EP000000211676,20/20,Newsmagazine
EP000000211683,20/20,Newsmagazine
EP000000211686,20/20,Newsmagazine
EP000000260120,227,Sitcom
EP000000351231,60 Minutes,Newsmagazine
EP000000351237,60 Minutes,Newsmagazine
EP000000351238,60 Minutes,Newsmagazine
EP000001830050,Alfred Hitchcock Hour,"Anthology,Drama,Suspense"


In [0]:

distinct_df.sort(f.col('prog_code').asc()).limit(150).show(150)

+--------------+--------------------+--------------------+
|     prog_code|               title|               genre|
+--------------+--------------------+--------------------+
|     DVRPGMKEY|                null|                null|
|EP000000211576|               20/20|        Newsmagazine|
|EP000000211614|               20/20|        Newsmagazine|
|EP000000211639|               20/20|        Newsmagazine|
|EP000000211645|               20/20|        Newsmagazine|
|EP000000211646|               20/20|        Newsmagazine|
|EP000000211647|               20/20|        Newsmagazine|
|EP000000211648|               20/20|        Newsmagazine|
|EP000000211650|               20/20|        Newsmagazine|
|EP000000211654|               20/20|        Newsmagazine|
|EP000000211659|               20/20|        Newsmagazine|
|EP000000211661|               20/20|        Newsmagazine|
|EP000000211665|               20/20|        Newsmagazine|
|EP000000211666|               20/20|        Newsmagazin

#Chat-GPT usage
GPT usage indicators are comments at the top of the block with #GPT.
Throughout the project, we used the chat mainly for sintax help, as our familiarity with spark sintax was minor at best, and resources for spark sintax on google are slim. Moreover, GPT was extremely useful for resolving errors, and understanding spark concepts. The prompts we used were for example:
"can you explain thoroughly how rank works?"
"window_spec = Window.partitionBy("DMA").orderBy(countDistinct(col("genre"))) is ths legal?"
"result_df = ProgramViewingData.withColumn("genre_count", count("*").over(window_spec))
where does it make the distinction between genres here"
"i need to filter rows and get all rows for col1 that their values are lower than the average of col1"